In [ ]:
import torch
import csv
import gzip
import nltk
from torch.utils.data import TensorDataset

In [ ]:
# Dictionary structure 
import gzip
import torch.nn as nn
from torch.autograd import Variable
from dpp_nets.utils.io import embd_iterator, make_embd
from collections import namedtuple
from nltk import word_tokenize
import torch
import numpy as np
from dpp_nets.my_torch.utilities import pad_tensor

id_to_content = {}
errors = []
#embd_path = '/Users/Max/data/askubuntu/vectors_stackexchange.txt.gz'
embd_path = '/Users/Max/data/askubuntu/vectors_pruned.200.txt.gz'
embd_layer, word_to_ix  = make_embd(embd_path)
question = namedtuple('question', 'id title body title_ix body_ix')
path = '/Users/Max/data/askubuntu/text_tokenized.txt.gz'

In [5]:
val_path = '/Users/Max/data/askubuntu/dev.txt'
count = 0
import torch

with open(val_path) as f:
    data_tensor = []
    target_tensor = []
    
    for line in f:
        q_id, pos, candidates, _ = line.split("\t")
        q_id = int(q_id)
        pos = [int(id) for id in pos.split()]
        candidates = [int(id) for id in candidates.split()]
        
        # Check
        if len(pos) == len(candidates):
            continue
        
        # Create target
        target = [1 if i in pos else 0 for i in candidates]
        target = [-1] + target
        target = torch.ByteTensor(target)
        

In [4]:
        # Create x Tensor 
        all_ids = [q_id] + candidates 
        all_ids = [id_to_content[id] for id in all_ids]
        all_ids = [pad_tensor(torch.LongTensor(np.array(id.title_ix)),0,0,max_title_size) for id in all_ids]
        all_ids = torch.stack(all_ids)
        data_tensor.append(all_ids)
        target_tensor.append(target)
        
    data_tensor = torch.stack(data_tensor)
    target_tensor = torch.stack(target_tensor)

NameError: name 'id_to_content' is not defined

In [8]:
val_ds.target_tensor.size()

torch.Size([193, 20])

In [9]:
val_ds.data_tensor.size()

torch.Size([193, 21, 38])

In [ ]:
embd_layer, word_to_ix  = make_embd(embd_path)

In [ ]:
id_to_content, word_to_ix, max_title_size, max_body_size = create_id_to_content(data_base_path, word_to_ix)

In [ ]:
embd = update_embd(embd_layer, word_to_ix)

In [ ]:
title_ds, _ = create_train_set(train_path, id_to_content, max_title_size, max_body_size)

In [ ]:
titles = []
bodies = []
targets = []

with open(train_path) as f:
    for line in f:
        q_id, pos, neg = line.split("\t")
        q_id = int(q_id)
        q_content = id_to_content[q_id]
        q_title = pad_tensor(torch.LongTensor(np.array(q_content.title_ix)),0,0, max_title_size)
        q_body = pad_tensor(torch.LongTensor(np.array(q_content.body_ix)), 0,0, max_body_size)
        
        pos = [id_to_content[int(id)] for id in pos.split()]
        neg = [id_to_content[int(id)] for id in neg.split()]
        
        targets = [for q in ]
        targets.extend([1] * len(pos))
        targets.extend([-1] * len(neg))

        pos_pairs_title = [torch.stack([q_title, pad_tensor(torch.LongTensor(np.array(q.title_ix)),0,0,max_title_size)]) 
                           for q in pos]
        neg_pairs_title = [torch.stack([q_title, pad_tensor(torch.LongTensor(np.array(q.title_ix)),0,0,max_title_size)])
                            for q in neg]

        titles.extend(pos_pairs_title)
        titles.extend(neg_pairs_title)        

        pos_pairs_body = [torch.stack([q_body, pad_tensor(torch.LongTensor(np.array(q.body_ix)),0,0,max_body_size)]) 
                           for q in pos]
        neg_pairs_body = [torch.stack([q_body, pad_tensor(torch.LongTensor(np.array(q.body_ix)),0,0,max_body_size)]) 
                           for q in neg]

        bodies.extend(pos_pairs_body)
        bodies.extend(neg_pairs_body)

In [1]:
embd_path = '/Users/Max/data/askubuntu/vectors_pruned.200.txt.gz'
data_base_path = '/Users/Max/data/askubuntu/text_tokenized.txt.gz'
train_path = '/Users/Max/data/askubuntu/train_random.txt'
val_path = '/Users/Max/data/askubuntu/dev.txt'

from dpp_nets.utils.ubuntu_io import create_ubuntu 

In [2]:
embd, train_ds, val_ds = create_ubuntu(embd_path, data_base_path, train_path, val_path)

In [ ]:
def train(loader, model, criterion, optimizer):

    model.train()

    for t, (review, target) in enumerate(loader):
        
        review = Variable(review)

        if args.aspect == 'all':
            target = Variable(target[:,:3])
        else:
            target = Variable(target[:,int(args.aspect[-1])])

        pred = model(review)
        loss = criterion(pred, target)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        print("trained one batch")

def validate(loader, model, criterion):

    model.eval()

    MAP = 0.0

    for i, (review, target) in enumerate(loader, 1):

        review = Variable(review, volatile=True)

        if args.aspect == 'all':
            target = Variable(target[:,:3], volatile=True)
        else:
            target = Variable(target[:,int(args.aspect[-1])], volatile=True)

        pred = model(review)
        loss = criterion(pred, target)
        
        delta = loss.data[0] - total_loss
        total_loss += (delta / i)

        print("validated one batch")
        
        

In [ ]:
import torch.nn as nn


In [ ]:
nn.functional.C

In [ ]:

embd_dim = embd.weight.size(1)
hidden_dim = 500
enc_dim = 200
target_dim = 50

# Model
torch.manual_seed(0)
net = DeepSetBaseline(embd_dim, hidden_dim, enc_dim, target_dim)
model = nn.Sequential(embd, net)
print("created model")

### Set-up training
criterion = nn.CosineEmbeddingLoss()
optimizer = torch.optim.Adam(net.parameters(), lr=0.003)
print("set up optimizer")

embd.requires_grad = False

In [ ]:
train_ds.data_tensor[1:10,0,:].size()

In [ ]:
train_loader = DataLoader(train_ds, 20, shuffle=True)

In [ ]:
model.train()

for t, (qs, target) in enumerate(train_loader):
    q1 = Variable(qs[:,0,:])
    q2 = Variable(qs[:,1,:])

    target = Variable(target)

    pred1 = model(q1)
    pred2 = model(q2)

    loss = criterion(pred1, pred2, target)
    loss.backward()
    print(loss.data)


In [ ]:
batch = Variable(qs[:,0,:])
words = embd(batch)
embd.weight.requires_grad = False

In [ ]:
embd_dim = 200
hidden_dim = 500
enc_dim = 200
target_dim = 50

batch_size, max_set_size, embd_dim = words.size()

# Unpacking to send through encoder network
# Register indices of individual instances in batch for reconstruction
lengths = words.sum(2).abs().sign().sum(1)
s_ix = list(lengths.squeeze().cumsum(0).long().data - lengths.squeeze().long().data)
e_ix = list(lengths.squeeze().cumsum(0).long().data)

# Filter out zero words 
mask = words.sum(2).abs().sign().expand_as(words).byte()
words = words.masked_select(mask).view(-1, embd_dim)



In [ ]:
def compute_ap(scores, target):
    
    if not target.sum():
        return 0 
    
    # sort the targets according to scores
    order = scores.argsort()
    copy = target.copy()
    target = copy[order]
    
    # compute average precision
    tp_k = target.cumsum() * target
    tot_k = np.arange(1, len(target) + 1)
    p_k = tp_k / tot_k
    ap = np.mean(p_k[p_k > 0])
    
    return ap

In [ ]:
import numpy
import numpy as np


scores = np.array([3.1, 1.2, 4.4, 2.1, 5.5])
target = np.array([0, 0, 0, 0, 1])
compute_ap(scores, target)